# Data Cleaning & Preprocessing (Classification Competition)
This notebook performs **competition-grade data cleaning** and exports a cleaned dataset that will be reused in later notebooks.

✅ Outputs:
- `data/cleaned_students.csv` (cleaned features + target class)
- `data/cleaning_report.json` (quick summary for reproducibility)

Target for classification will be created from `exam_score` as `exam_score_class`.
- **Low**: < 50
- **Medium**: 50 to 75 (inclusive)
- **High**: > 75

> You can change thresholds in the `TARGET_BINNING` cell if your competition uses different labels.

## 0) Setup
This section imports libraries and sets notebook-wide options.


In [ ]:
# Core imports
import os
from pathlib import Path
import json
import numpy as np
import pandas as pd

# Display options
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)
pd.set_option("display.max_rows", 80)

RANDOM_STATE = 42

print("✅ Setup complete")

## 1) Load Data
Update `DATA_PATH` if your dataset file name/location is different.

**Recommended folder structure**:
- `data/raw_students.csv` (input)
- `data/cleaned_students.csv` (output)


In [ ]:
# === Update this path if needed ===
DATA_PATH = Path("data/raw_students.csv")

# Output paths
OUT_DIR = Path("data")
OUT_DIR.mkdir(parents=True, exist_ok=True)

CLEAN_PATH = OUT_DIR / "cleaned_students.csv"
REPORT_PATH = OUT_DIR / "cleaning_report.json"

if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"❌ Could not find file at: {DATA_PATH}\n"
        f"➡️ Place your dataset there OR update DATA_PATH accordingly."
    )

df = pd.read_csv(DATA_PATH)
print("✅ Loaded:", DATA_PATH)
print("Shape:", df.shape)
df.head()

## 2) Quick Audit
We check columns, dtypes, missing values, duplicates, and basic statistics.

In [ ]:
print("Columns:", list(df.columns))
display(df.info())

missing = df.isna().sum().sort_values(ascending=False)
display(pd.DataFrame({"missing": missing, "missing_%": (missing/len(df)*100).round(3)}).head(20))

dup_count = df.duplicated().sum()
print("Duplicate rows:", dup_count)

display(df.describe(include="all").T)

## 3) Standardize Column Names (safe)
We keep your original column names if already clean, but we also ensure no accidental spaces.

In [ ]:
df.columns = [c.strip() for c in df.columns]

# Expected columns based on your sample description
EXPECTED_COLS = [
    "student_id", "age", "gender", "course", "study_hours", "class_attendance",
    "internet_access", "sleep_hours", "sleep_quality", "study_method",
    "facility_rating", "exam_difficulty", "exam_score"
]

missing_cols = [c for c in EXPECTED_COLS if c not in df.columns]
extra_cols = [c for c in df.columns if c not in EXPECTED_COLS]

print("Missing expected columns:", missing_cols)
print("Extra columns (allowed):", extra_cols)

## 4) Fix Categorical Text (lowercase, strip)
This avoids accidental duplicates like `'Male'` vs `'male '`.

In [ ]:
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
print("Categorical cols detected:", cat_cols)

def normalize_text(s: pd.Series) -> pd.Series:
    return (s.astype(str)
            .str.strip()
            .str.lower()
            .replace({"nan": np.nan, "none": np.nan, "": np.nan})
           )

for c in cat_cols:
    df[c] = normalize_text(df[c])

# Optional: show unique values (top)
for c in cat_cols:
    uniques = df[c].dropna().unique()
    print(f"\n{c} uniques ({len(uniques)}): {uniques[:30]}")

## 5) Validate & Repair Numeric Ranges
Competition-friendly cleaning: we **cap** outliers rather than dropping many rows.

We apply sensible domain constraints:
- `age`: 15–30 (sample indicates 17–24)
- `study_hours`: 0–16 (sample indicates ~0–8)
- `class_attendance`: 0–100
- `sleep_hours`: 0–24 (sample indicates ~4–10)
- `exam_score`: 0–100


In [ ]:
num_cols = df.select_dtypes(include=["int64", "float64", "int32", "float32"]).columns.tolist()
print("Numeric cols detected:", num_cols)

RANGES = {
    "age": (15, 30),
    "study_hours": (0, 16),
    "class_attendance": (0, 100),
    "sleep_hours": (0, 24),
    "exam_score": (0, 100),
}

range_report = {}
for col, (lo, hi) in RANGES.items():
    if col not in df.columns:
        continue
    before_invalid = ((df[col] < lo) | (df[col] > hi)).sum()
    df[col] = df[col].clip(lo, hi)
    after_invalid = ((df[col] < lo) | (df[col] > hi)).sum()
    range_report[col] = {"clipped_before": int(before_invalid), "clipped_after": int(after_invalid)}

pd.DataFrame(range_report).T

## 6) Handle Missing Values
Your sample suggests there are no missing values, but competitions can include them.

- Numeric: fill with median
- Categorical: fill with mode (`most frequent`)


In [ ]:
missing_before = df.isna().sum().sum()
print("Total missing values (before):", int(missing_before))

# Identify columns by type
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
num_cols = [c for c in df.columns if c not in cat_cols]

# Fill numeric
for c in num_cols:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].median())

# Fill categorical
for c in cat_cols:
    if df[c].isna().any():
        mode = df[c].mode(dropna=True)
        fill_val = mode.iloc[0] if len(mode) else "unknown"
        df[c] = df[c].fillna(fill_val)

missing_after = df.isna().sum().sum()
print("Total missing values (after):", int(missing_after))

## 7) Remove Duplicates (if any)
We drop exact duplicates. `student_id` duplicates are handled separately if present.

In [ ]:
dup_rows = df.duplicated().sum()
print("Exact duplicate rows:", int(dup_rows))

if dup_rows > 0:
    df = df.drop_duplicates().reset_index(drop=True)

print("Shape after dropping exact duplicates:", df.shape)

## 8) Student ID Sanity
`student_id` is an identifier and will be excluded from modeling later, but we keep it here.
If duplicates exist, we keep the first occurrence.

In [ ]:
if "student_id" in df.columns:
    dup_id = df["student_id"].duplicated().sum()
    print("Duplicate student_id count:", int(dup_id))
    if dup_id > 0:
        df = df.drop_duplicates(subset=["student_id"]).reset_index(drop=True)
        print("✅ Dropped duplicate student_id rows. New shape:", df.shape)
else:
    print("⚠️ student_id column not found (ok if your dataset differs).")

## 9) Create Classification Target (`exam_score_class`)
Since the project is for **classification**, we convert numeric `exam_score` into discrete classes.

Default bins:
- Low: `< 50`
- Medium: `50–75`
- High: `> 75`

If your competition provides specific class labels, update the thresholds here.

In [ ]:
# === Adjust thresholds here if needed ===
LOW_MAX = 50
HIGH_MIN = 75

if "exam_score" not in df.columns:
    raise ValueError("❌ 'exam_score' column not found. Please verify your dataset columns.")

def score_to_class(x: float) -> str:
    if x < LOW_MAX:
        return "low"
    elif x <= HIGH_MIN:
        return "medium"
    else:
        return "high"

df["exam_score_class"] = df["exam_score"].apply(score_to_class)

# Distribution
dist = df["exam_score_class"].value_counts()
dist_pct = (dist / len(df) * 100).round(2)
display(pd.DataFrame({"count": dist, "percent": dist_pct}))

df[["exam_score", "exam_score_class"]].head(10)

## 10) Final Type Checks + Export
We export the cleaned dataset for downstream steps.

Saved file will be used in:
- `eda.ipynb`
- `feature_engineering.ipynb`
- `model.ipynb`


In [ ]:
# Optional: enforce expected dtypes (safe casting)
# Note: We do not cast aggressively if columns are missing.
dtype_plan = {
    "student_id": "int64",
    "age": "int64",
    "study_hours": "float64",
    "class_attendance": "float64",
    "sleep_hours": "float64",
    "exam_score": "float64",
}

for col, dt in dtype_plan.items():
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype(dt)

# final missing check
assert df.isna().sum().sum() == 0, "❌ Missing values still present after cleaning."

# Save cleaned dataset
df.to_csv(CLEAN_PATH, index=False)
print("✅ Cleaned dataset saved to:", CLEAN_PATH)

# Save quick report
report = {
    "input_path": str(DATA_PATH),
    "output_path": str(CLEAN_PATH),
    "rows": int(df.shape[0]),
    "cols": int(df.shape[1]),
    "duplicate_rows_removed": int(dup_rows),
    "range_clipping": range_report,
    "target_binning": {"low_max_exclusive": LOW_MAX, "high_min_inclusive": HIGH_MIN},
    "target_distribution": df["exam_score_class"].value_counts().to_dict(),
    "columns": list(df.columns),
}

with open(REPORT_PATH, "w", encoding="utf-8") as f:
    json.dump(report, f, indent=2)

print("✅ Cleaning report saved to:", REPORT_PATH)

df.head()

## 11) Quick Smoke Tests
These checks ensure the exported dataset is stable for later steps.

In [ ]:
# Reload to verify
df_check = pd.read_csv(CLEAN_PATH)

print("Reloaded shape:", df_check.shape)
print("Target classes:", df_check["exam_score_class"].unique())

# Basic sanity
assert set(df_check["exam_score_class"].unique()).issubset({"low", "medium", "high"})
assert df_check["exam_score"].between(0, 100).all()

print("✅ Smoke tests passed")